In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
# Define the file paths for the dataset
train_dir = '/content/gdrive/MyDrive/asl_alphabet_train'

In [4]:
test_dir = '/content/gdrive/MyDrive/asl_alphabet_test'

In [5]:
# Create a list of folders' names for each ASL symbol
folder = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','del','nothing','space']

In [6]:
# Import packages and set numpy random seed
import random
import numpy as np
from os.path import join
from os import listdir
import matplotlib.pyplot as plt
import cv2
import skimage
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import image

np.random.seed(5) 
tf.random.set_seed(2)
%matplotlib inline

# Define functions to load, shuffle, and split the data
def load_data(container_path=train_dir, folders=folder,
              size=87000, test_split=0.2, seed=0): # 3000 images in 29 classes
    """
    Loads sign language dataset.
    """
    
    filenames, labels = [], []

    for label, folder in enumerate(folders):
        folder_path = join(container_path, folder)
        images = [join(folder_path, d)
                     for d in sorted(listdir(folder_path))]
        labels.extend(len(images) * [label])
        filenames.extend(images)
    
    random.seed(seed)
    data = list(zip(filenames, labels))
    random.shuffle(data)
    data = data[:size]
    filenames, labels = zip(*data)

     # Get the images
    x = paths_to_tensor(filenames).astype('float32')/255
    # Store the one-hot targets
    y = np.array(labels)

    x_train = np.array(x[:int(len(x) * (1 - test_split))])
    y_train = np.array(y[:int(len(x) * (1 - test_split))])
    x_test = np.array(x[int(len(x) * (1 - test_split)):])
    y_test = np.array(y[int(len(x) * (1 - test_split)):])
     
    print('Loaded', len(x_train),'images for training,','Train data shape =',x_train.shape)
    print('Loaded', len(x_test),'images for testing','Test data shape =',x_test.shape)
    print('Loaded', len(x_train),'labels for training,','Train data shape =',y_train.shape)
    print('Loaded', len(x_test),'labels for testing','Test data shape =',y_test.shape)
    return (x_train, y_train), (x_test, y_test)


def path_to_tensor(img_path, size):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(size, size))
    # convert PIL.Image.Image type to 3D tensor
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor 
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths, size=50):
    list_of_tensors = [path_to_tensor(img_path, size) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [ ]:
# Load pre-shuffled training and test datasets
(x_train, y_train), (x_test, y_test) = load_data()